In [65]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
import parseIntervalFiles as pif
import parseActivityFiles as paf
#pun intended :)
import consolidateFiles as cf
import hervpd as hp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0 - Pipeline configuration 
* set the input/output directories, user id and verbose level

In [52]:
    verbose = True    
    user = 0
    
    RAW_DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/Raw/%d"%user
    PRE_DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/PreProcessed/%d"%user
    
    # duration (in seconds) to be cropped from the beginning of each second to account for stabilization and user adjustment
    crop = 90
    
    # duration (in seconds) of each fragment to be sent to analysis
    duration = 300
    
    # if any fragment has more than 'threshold' consecutive seconds with no beats, it will be discarded
    threshold = 3

## 1 - Extract sessions from raw data (intervals + annotations files)

### 1.1 - List activities with session start/stop datetime
Parses activities files and prints all errors found (activities without start or stop, invalid dates, etc)

In [53]:
sessions = paf.parseActivityFiles(dirname=RAW_DATA_PATH)

reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171014.csv ... 
orphan stop in: ['2017-10-14 2:14:21', 'stop', '', '', '']
orphan start in: ['2017-10-14 11:58:31', 'start', 'focused-active', 'lying down', '']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171102.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171016.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171006.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171104.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171029.csv ... 
orphan stop in: ['2017-10-29 15:27:59', 'stop', '', '']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171028.csv ... 
orphan start in: ['2017-10-28 11:48:07', 'start', 'sleep', 'posture']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171013.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171003.csv ... 
orphan start in: ['2017-10-03 22:56:19', 'start', 'eat', 'sitting']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/

## 2 - Generate fragments from sessions

### 2.1 - breaks the sessions duration in fragments
Configurations:
* duration of each fragment in seconds;
* number of seconds to be discarded at the beginning of the session, accounting for user's stabilization and adjustment to posture and activity 

### 2.2 - extracts the intervals for each fragment
Retrieves from the heartbeat files all the intervals contained in each session's duration and adds them to the fragments objects (in memory)


In [54]:
frags = cf.fragment_sessions(sessions, duration, crop)

122 valid sessions out of 157 total (at least one full fragment of 300 seconds after discarding first 90 seconds)


## 3 - Quality assurance

### To be implemented


In [55]:
valid_frags = [f for f in frags if len(cf.beats_in_fragment(f, RAW_DATA_PATH)) > 0.8*duration ]

## 4 - Extract features

In [56]:
beats = cf.beatlist(cf.beats_in_fragment(frags[100], RAW_DATA_PATH))
print(cf.calc_metrics(beats))

{'rmssd': 45.477477553512408, 'sdnn': 68.814471910244038, 'nn50': 16, 'pnn50': 2.572347266881029, 'mrri': 490.03049759229532, 'mhr': 124.57569788680118, 'total_power': 2102.5544791818802, 'vlf': 379.11538427722235, 'lf': 865.19516073512818, 'hf': 858.24393416952989, 'lf_hf': 1.0080993599708044, 'lfnu': 50.201667311196253, 'hfnu': 49.798332688803761}


In [60]:
print(len(valid_frags))
df = [cf.aggregate_data(f, RAW_DATA_PATH) for f in valid_frags]

1286


In [77]:
pdf = pd.DataFrame(df)
pdf.head()

,activity,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,order,pnn50,rmssd,sdnn,sess,start,stop,total_power,vlf
0,focused-active,1516.951144,60.039661,1009.630650,0.665566,39.960339,77.562161,786.159383,106,0,27.319588,64.259882,98.632350,0,2017-10-14 02:15:51,2017-10-14 02:20:51,4622.061918,2095.480124
1,focused-active,443.401880,47.028536,499.433927,1.126369,52.971464,76.262208,791.421189,60,1,15.544041,38.154071,59.187774,0,2017-10-14 02:20:51,2017-10-14 02:25:51,2122.588509,1179.752702
2,focused-active,556.500976,47.711706,609.881487,1.095922,52.288294,75.781084,795.275325,50,2,13.020833,38.068024,53.117372,0,2017-10-14 02:25:51,2017-10-14 02:30:51,1818.107217,651.724754
3,focused-active,2556.565157,60.438465,1673.464748,0.654575,39.561535,73.561917,828.932249,136,3,36.956522,86.832813,109.005825,0,2017-10-14 02:30:51,2017-10-14 02:35:51,6779.012728,2548.982823
4,focused-active,764.052485,41.364174,1083.083368,1.417551,58.635826,76.085082,794.917313,93,4,24.093264,47.554632,71.367134,0,2017-10-14 02:35:51,2017-10-14 02:40:51,3062.008804,1214.872951


In [75]:
#pdf.describe()
#r = hp.runFlow(pdf)
hp.scaleFeatures(pdf)

/home/ju/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/ju/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/ju/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype datetime64[ns] was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/ju/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [78]:
pdf.describe()

,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,order,pnn50,rmssd,sdnn,sess,total_power,vlf
count,1.286000e+03,1286.000000,1.286000e+03,1286.000000,1286.000000,1286.000000,1286.000000,1286.000000,1286.000000,1286.000000,1286.000000,1286.000000,1286.000000,1.286000e+03,1.286000e+03
mean,2.525854e+04,55.341049,2.886178e+05,1.754342,44.658951,82.380120,789.116547,117.090980,24.825816,32.470789,132.776426,125.719080,56.260498,5.821772e+05,2.683009e+05
std,1.539959e+05,23.850139,4.889699e+06,4.007776,23.850139,19.229308,171.501216,82.508843,27.996067,26.415424,140.785066,102.525861,35.567131,1.023972e+07,5.261225e+06
min,2.147458e+00,1.456776,1.739058e+01,0.027289,2.656368,49.448981,354.596065,0.000000,0.000000,0.000000,3.369205,13.625541,0.000000,1.108732e+02,3.704196e+01
25%,5.267165e+02,37.248277,6.100922e+02,0.332417,24.948430,67.332358,665.723620,40.000000,3.000000,8.747651,39.129113,57.077112,23.000000,1.842039e+03,5.190481e+02
50%,2.206278e+03,58.587815,1.500708e+03,0.706840,41.412185,80.087278,764.922500,105.000000,12.000000,24.159042,89.425239,99.076177,58.000000,5.267396e+03,1.204442e+03
75%,1.358343e+04,75.051570,4.183391e+03,1.684689,62.751723,92.346774,920.116568,199.000000,40.000000,55.496280,194.099499,171.846782,78.000000,2.330235e+04,2.996180e+03
max,3.481862e+06,97.343632,1.657137e+08,67.644710,98.543224,169.796600,1240.951417,354.000000,113.000000,91.726619,1667.724397,1226.359214,121.000000,3.491670e+08,1.799714e+08


In [80]:
pdf.loc[pdf.hf > 1000000]

,activity,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,order,pnn50,rmssd,sdnn,sess,start,stop,total_power,vlf
113,household-chores,1.106908e+06,22.782275,3.751727e+06,3.389377,77.217725,103.029756,655.965739,75,6,16.094421,711.716355,545.558334,9,2017-10-14 13:48:28,2017-10-14 13:53:28,6.859219e+06,2.000584e+06
537,sleep,2.474411e+06,7.423667,3.085697e+07,12.470431,92.576333,79.669903,852.571031,168,49,46.927374,460.809232,397.942859,42,2017-11-09 04:30:32,2017-11-09 04:35:32,4.660044e+07,1.326907e+07
555,sleep,1.122949e+06,3.425050,3.166339e+07,28.196653,96.574950,77.857616,953.015528,247,67,76.947040,1097.452557,767.879751,42,2017-11-09 06:00:32,2017-11-09 06:05:32,7.195909e+07,3.917275e+07
1085,eat,1.216702e+06,19.404664,5.053449e+06,4.153400,80.595336,101.162743,965.132911,185,6,58.730159,1479.885082,1048.500217,106,2017-10-04 13:10:04,2017-10-04 13:15:04,7.482753e+06,1.212602e+06
1086,eat,1.176622e+06,23.417895,3.847834e+06,3.270239,76.582105,117.007072,811.032172,220,7,59.139785,1199.368790,872.067106,106,2017-10-04 13:15:04,2017-10-04 13:20:04,6.758502e+06,1.734047e+06
1087,eat,1.149476e+06,35.161260,2.119679e+06,1.844039,64.838740,107.797894,743.728155,184,8,44.768856,785.166743,625.058490,106,2017-10-04 13:20:04,2017-10-04 13:25:04,3.772668e+06,5.035132e+05
1098,eat,1.619905e+06,7.482271,2.003000e+07,12.364926,92.517729,116.145955,642.578616,158,19,33.193277,1024.735296,724.396072,106,2017-10-04 14:15:04,2017-10-04 14:20:04,2.998887e+07,8.338966e+06
1150,focused-passive,1.067715e+06,26.583970,2.948672e+06,2.761665,73.416030,101.180399,843.151934,209,39,57.894737,972.770573,788.434939,107,2017-10-05 12:51:13,2017-10-05 12:56:13,5.482440e+06,1.466053e+06
1220,leisure-passive,3.481862e+06,2.057892,1.657137e+08,47.593423,97.942108,114.527201,755.661728,196,9,48.514851,1667.724397,1226.359214,120,2017-09-29 20:05:28,2017-09-29 20:10:28,3.491670e+08,1.799714e+08
